In [13]:
import random
import networkx  
import pgmpy.models as pgmm  
import pgmpy.factors.discrete as pgmf              
import pgmpy.inference as pgmi
import random
import numpy
import time
import re
import Utilidades as ut

mina = '☠'
casilla_sin_pulsar = '□'

class Juego:
    def __init__(self, filas, columnas,minas):
        self.filas = filas
        self.columnas = columnas
        self.minas = minas
        self.tablero_buscaminas_oculto = ut.ocultar_casillas_tablero (filas, columnas)
        self.tableroBuscaminas = ut.tablero_buscaminas (filas, columnas, minas)

class Buscaminas_bayesiano(Juego):
    def __init__(self,filas,columnas,minas):
        super().__init__(filas, columnas, minas)
        self.Minas_encontradas = []
        
    def red_bayesiana(self):
        
        """
            Creamos la red bayesiana con la librería pgmpy
        """
        Buscaminas_bayesiano = pgmm.BayesianModel()

        """
            Variables importantes:
            · x_totales: casillas ocultas totales del tablero
            · x_ocultas: casillas ocultas con vecinos destapados
            · y_destapadas: casillas destapadas
        """



        x_totales = []
        x_ocultas = []
        y_destapadas = []
        casilla_oculta_sin_colindantes_destapados = False

        """
            Añadimos los nodos a la red.
            Serán las casillas ocultas con vecinos destapados y las casillas destapadas.
        """

        for fila in range(self.filas):
            for columna in range(self.columnas):

                # Obtenemos la posicion de las casillas vecinas
                casillas_vecinas = ut.posicion_casillas_vecinas(fila,columna,self.tableroBuscaminas)

                # Si la casilla no está pulsada
                if (self.tablero_buscaminas_oculto[fila][columna] == casilla_sin_pulsar):

                    x = 'Tapada({0}{1})'.format(fila,columna) # Lo pone en formato Tapada (01),Tapada (02)...
                    x_totales.append(x) # Añadimos las casillas sin pulsar a la variable x_totales
                    casilla_tiene_vecinos_destapados = False

                    # Recorro todas las casillas vecinas de la casilla en la que estamos actualmente
                    for vecino in casillas_vecinas:
                        # Comprobamos que los vecinos de la casilla estan destapados
                        if(self.tablero_buscaminas_oculto[int(vecino[0])][int(vecino[1])] != casilla_sin_pulsar):
                            casilla_tiene_vecinos_destapados = True
                            break
                     # Comprobamos que la casilla actual tiene vecino destapado
                    if(not casilla_oculta_sin_colindantes_destapados or casilla_tiene_vecinos_destapados):
                        # Si lo tiene, añadimos la casilla a la lista de casillas tapadas con vecinos destapados
                        x_ocultas.append(x)
                        Buscaminas_bayesiano.add_nodes_from([x]) # Se añade la casilla como nodo de nuestra red

                    if(not casilla_tiene_vecinos_destapados):
                        casilla_oculta_sin_colidantes_destapados = True

                # Si la casilla está pulsada
                else:
                    y = 'Destapada({0}{1})'.format(fila,columna) # lo pone en formato Destapada (01),Destapada (02)...
                    casilla_tiene_vecinos_ocultos = False
                    #Listado_de_minas = listado_de_minas(tableroBuscaminas) # Variable que contiene la posición de las minas del tablero

                    for vecino in casillas_vecinas:
                        casilla_vecina_oculta = 'Tapada({0}{1})'.format(vecino[0],vecino[1])
                        if(self.tablero_buscaminas_oculto[int(vecino[0])][int(vecino[1])] == casilla_sin_pulsar and
                           not self.Minas_encontradas.__contains__(casilla_vecina_oculta)):
                            casilla_tiene_vecinos_ocultos = True
                            break

                    if(casilla_tiene_vecinos_ocultos):
                        y_destapadas.append(y)
                        Buscaminas_bayesiano.add_nodes_from([y])

        print("Lista Completa casillas ocultas : \n")
        print(x_totales)
        print("\n")
        print("Lista casillas vecinas ocultas acumuladas hasta la posicion actual : \n")
        print(x_ocultas)
        print("\n")
        print("Lista casillas destapadas hasta ese momento : \n")
        print(y_destapadas)
        print("\n")

        print("Comprobamos la lista de nodos añadidos a la red bayesiana:\n")
        print(Buscaminas_bayesiano.nodes())
        print("\n")

        print("Lista de casillas ocultas que son minas:\n")
        print(self.Minas_encontradas)
        print("\n")

        """
            Añadimos las aristas entre los nodos.
            Las casillas destapadas tendrán aristas con sus vecinas ocultas
        """

        Aristas = dict()
        ListaColindantesOcultos = []
        for fila in range(self.filas):
            for columna in range(self.columnas):
                Destapada = 'Destapada({0}{1})'.format(fila,columna)

                # Si la casilla ha sido destapada y se ha añadido a la lista de casillas destapadas
                if(self.tablero_buscaminas_oculto[fila][columna] != casilla_sin_pulsar
                      and y_destapadas.__contains__(Destapada)):

                    # Obtenemos la posicion de las casillas vecinas
                    casillas_vecinas = ut.posicion_casillas_vecinas(fila,columna,self.tableroBuscaminas)

                    # Recorremos sus vecinos
                    for vecino in casillas_vecinas: # Recorro sus vecinos
                        Tapada = 'Tapada({0}{1})'.format(vecino[0],vecino[1])

                        # Si el vecino es oculto añadimos la arista de la destapada a la tapada
                        if(x_ocultas.__contains__(Tapada)):
                            Buscaminas_bayesiano.add_edges_from([(Tapada,Destapada)])
                            ListaColindantesOcultos.append(Tapada)

                    # Asociamos la casilla destapada con todos sus vecinos ocultos  
                    Aristas.update({Destapada: ListaColindantesOcultos})

        print("Lista de aristas:\n")
        print(Aristas)
        print("\n")
        print("Lista de vecinos ocultos:\n")
        print(ListaColindantesOcultos)
        print("\n")
        print("Lista de aristas añadidas a la red bayesiana:\n") 
        print(Buscaminas_bayesiano.edges())
        print("\n")

        """
            Generación de tablas de probabilidad (CPDs)
            · Las tablas de probabilidad de las casillas tapadas no dependen de nada.
            · Las tablas de probabilidad de las casillas destapadas dependen de sus casillas vecinas tapadas.
        """

        """
            Para cada variable creamos una instancia de la clase `TabularCPD`, proporcionando:
            · Nombre de la variable (atributo `variable`),
            · Cantidad de valores (cardinalidad) que puede tomar (atributo `variable_card`),
            · Lista de listas, conteniendo cada una de estas las probabilidades para un valor concreto, 
              según los valores de los padres (atributo `values`, el valor se transforma a un array),
            · Lista con los nombres de los padres (atributo `evidence`, valor `None` si no se proporciona),
            · Lista con la cantidad de valores (cardinalidad) que puede tomar cada uno de los padres (atributo `evidence_card`, 
              valor `None` si no se proporciona).
        """

        """
            Tabla de probabilidad de las casillas tapadas
        """

        ListaValoresDestapados=dict()
        for fila in range(self.filas):
            for columna in range(self.columnas):
                Tapada = 'Tapada({0}{1})'.format(fila,columna)
                Destapada = 'Destapada({0}{1})'.format(fila,columna)
                if(self.tablero_buscaminas_oculto[fila][columna]==casilla_sin_pulsar and x_ocultas.__contains__(Tapada)):

                    # Si la casilla por detapar es una mina
                    if(self.Minas_encontradas.__contains__(Tapada)):
                        probabilidad_mina = 1

                    # Si el numero de casillas por destapar es mayor que el numero de minas que tiene el tablero
                    elif(len(x_ocultas)>0):
                        #numero de casillas destapadas = (X*Y)-len(x_ocultas)
                        probabilidad_mina = (self.minas-len(self.Minas_encontradas))/len(x_totales)
                    else:
                        probabilidad_mina = 0

                    # Añadimos la cpd al modelo
                    cpd = pgmf.TabularCPD(Tapada, 2, [[1-probabilidad_mina,probabilidad_mina]])
                    Buscaminas_bayesiano.add_cpds(cpd)


                # CPD de Destapadas
                elif(self.tablero_buscaminas_oculto[fila][columna] != casilla_sin_pulsar and y_destapadas.__contains__(Destapada)):
                    ListaValoresDestapados.update({Destapada: int(self.tablero_buscaminas_oculto[fila][columna])})

                   #A continuación divido entre los valores que puede tener 'Destapada' en función de las minas que tenga alrededor
                    nColumnasCPD = pow(2, len(Aristas[Destapada]))
                    numero_maximo_minas = 9

                    # Si no toca ninguna pared como máximo 8 minas
                    # Si toca un lado como máximo 5 minas
                    # Si toca 2 lados (está en una esquina) como máximo 3 minas
                    # if((i == 0 and j == 0) or (i==X and j == Y) or (i == 0 and j == Y) or (i==X and j ==0)): #esto es una esquina
                    #    numeroMaximoMinas = 3
                    # if((i == 0 and (j > 0 and j < Y)) or (j == 0 and (i > 0 and i < X)) or (i==X and (j > 0 and j < Y)) or (j==Y and (i > 0 and i < X))): #esto es 1 lado
                    #    numeroMaximoMinas = 5
                    # else:
                    #    numeroMaximoMinas = 8


                    if(fila==0 or fila==self.filas-1):
                        numero_maximo_minas = 6
                    if(columna==0 or columna==self.columnas-1):
                        if(numero_maximo_minas == 6):
                            numero_maximo_minas = 4
                        else:
                            numero_maximo_minas = 6
                    if(len(Aristas[Destapada])+1 < numero_maximo_minas):
                        numero_maximo_minas = len(Aristas[Destapada])+1

                    listaProb = []
                    for numero_minas in range(numero_maximo_minas):
                        contador = 0
                        listaFila = []
                        for x in range(nColumnasCPD):
                            if(numero_minas == bin (contador).count('1')):
                                listaFila.append(1)
                            else:
                                listaFila.append(0)
                            contador = contador + 1

                        listaProb.append(listaFila)

                    ListaCardinalidad = []
                    for _ in range(len(Aristas[Destapada])):
                        ListaCardinalidad.append(2)

                    cpd = pgmf.TabularCPD(Destapada, numero_maximo_minas, listaProb, Aristas[Destapada], ListaCardinalidad)

                    Buscaminas_bayesiano.add_cpds(cpd)

                    print("\nIMPRIMO TODOS LOS FACTORES DE LA CPD: \n")

                    #Cada columna arriba, indica los padres de la variable que se calcula la probabilidad.
                    #for m in Buscaminas_bayesiano.cpds: print(m)

        Buscaminas_bayesiano_ev = pgmi.VariableElimination(Buscaminas_bayesiano) 
        consultas = []
        for variable in x_ocultas:
            if(not variable in self.Minas_encontradas):
                consulta = Buscaminas_bayesiano_ev.query([variable], ListaValoresDestapados)
                consultas.append(consulta)

        ocultasProbabilidadDeNoSerMina = dict()
        for variable in consultas:
            x = ''
            probabilidadDeNoSerMina = 0
            for variable1 in variable.keys():
                x = str(variable1)
            for variable2 in variable.values():
                probabilidadDeNoSerMina = float(variable2.values[0])

            if(probabilidadDeNoSerMina == 0.0):
                self.Minas_encontradas.append(x)


            ocultasProbabilidadDeNoSerMina.update({x: probabilidadDeNoSerMina}) 

        maximaProbabilidadDeNoSerMina = 0.
        ocultaMaximaProbabilidadDeNoSerMina = ''
        mejor_oculta = []
        print("Mejor casilla oculta con alta probabilidad de no ser mina:\n")
        for x, probabilidadDeNoSerMina in ocultasProbabilidadDeNoSerMina.items():
            print("x: {0} -> Probabilidad de no ser mina: {1}".format(x, probabilidadDeNoSerMina))
            if (maximaProbabilidadDeNoSerMina < probabilidadDeNoSerMina):
                ocultaMaximaProbabilidadDeNoSerMina = str(x)
                maximaProbabilidadDeNoSerMina = probabilidadDeNoSerMina
                mejor_oculta = []
                mejor_oculta.append(ocultaMaximaProbabilidadDeNoSerMina)

            """
            if (maximaProbabilidadDeNoSerMina == probabilidadDeNoSerMina and not x in mejor_oculta):
                mejor_oculta.append(x)
            """
            if (probabilidadDeNoSerMina == 1.0 and not x in mejor_oculta):
                mejor_oculta.append(x)

        
        """
        print("\nCasillas que son mina:\n")
        for y in Minas_encontradas:
            print("Mina: {0} -> Probabilidad de no ser mina: 0.000000000".format(y))
        """
        # Refreshing list of best_x to not be mine
        self.mejor_oculta = mejor_oculta
        print("\nLista de mejores casillas ocultas con probabilidad de no ser minas: {0} ".format(self.mejor_oculta))
        print(self.mejor_oculta)

SyntaxError: invalid syntax (<ipython-input-13-0020966d95af>, line 10)